In [91]:
import pandas as pd
from music21 import converter, instrument, note, chord
from sklearn.preprocessing import LabelEncoder
import itertools
import warnings
warnings.filterwarnings("ignore")

In [92]:
path = 'data\maestro-v3.0.0.csv'
df = pd.read_csv(path)

In [93]:
df = df.sort_values(by='duration').iloc[:10]

In [94]:
def extraction_notes(file):
    midi = converter.parse(file)
    L_notes = []
    for e in midi.recurse().notes:
        if e.isNote:
            x = [e.pitch.midi]+9*[-1]
            L_notes.append(x)
        elif e.isChord:
            x = [p.midi for p in e.pitches]
            while len(x)<10:
                x.append(-1)
            L_notes.append(x)
    if len(L_notes)<200:
        return 'null'
    else:
        return L_notes

def ex_notes_tab(df,col_out):
    df['midi_file_data'] = 'data/'+df['midi_filename']
    df[col_out] = df['midi_file_data'].apply(extraction_notes)
    return df[df[col_out]!='null']

def colonnes(df):
    df_expanded = df['X'].apply(pd.Series)
    df_expanded['y'] = df['y']

    return df_expanded

def sequence_list(L):
    sequence_length = 200

    input_sequences = []
    output_notes = []

    for i in range(len(L) - sequence_length):
        seq_in = L[i:i + sequence_length]  
        seq_out = L[i + sequence_length]  
        input_sequences.append(seq_in)
        output_notes.append(seq_out)

    return input_sequences,output_notes

def sequence(dff):
    X,y = [],[]
    for k in dff['notes'].to_list():
        xtemp,ytemp = sequence_list(k)
        X.append(xtemp)
        y.append(ytemp)
    X=list(itertools.chain(*X))
    y=list(itertools.chain(*y))

    return pd.DataFrame({"X":X,"y":y})
    

In [95]:
def preprocessing(df,split):
    dff = df[df['split']==split]
    #print(dff.head())
    dff = ex_notes_tab(dff,'notes')
    #print(dff.head())
    return colonnes(sequence(dff))

In [96]:
for k in df['split'].unique():
    print(k)
    dfp = preprocessing(df,k)
    dfp.to_csv('C:/Users/ccdeb/Documents/CS/3A/DEEPL/PROJET/data/'+k+'_10_V2.csv',index=False)

validation
train
test
